In [1]:
import treetime
import matplotlib.pyplot as plt
from Bio import Phylo
import os

tree_path = "../../docs/results/latest/branch_support/branch_support.nwk"
metadata_path = "../../docs/results/latest/metadata/metadata.tsv"
#metadata_path = "test/test.metadata.tsv"
outdir = "../../docs/results/latest/timetree/"

plt.rc('lines', linewidth=1)
dpi = 400

DATE_COL = "Date"
NAME_COL = "Sample"
REF_DATE = 1992.0
REF_LEN = 4653728
N_IQD=3

# Create a basic TimeTree Object
dates = treetime.utils.parse_dates(metadata_path, 
                                   date_col=DATE_COL, 
                                   name_col = NAME_COL)
dates["Reference"] = REF_DATE

tt = treetime.TreeTime(dates=dates, 
                           tree=tree_path, 
                           #gtr="infer", 
                           seq_len=REF_LEN,
                           verbose=4, 
                           fill_overhangs=False)

# Remove outliers
tt.clock_filter(reroot=None, 
                n_iqd=N_IQD, 
                plot=False)

# Root to tip regression
fig, (ax1) = plt.subplots(1, dpi=400)
tt.plot_root_to_tip(
    ax=ax1,
    )
ax1.set_xlabel("Date")
ax1.set_ylabel("Root-to-Tip Distance")

out_path = os.path.join(outdir, "timetree_rtt.jpg") 
plt.savefig(out_path, 
            dpi=dpi, 
            bbox_inches = "tight")


Attempting to parse dates...

	ERROR: file ../../docs/results/latest/metadata/metadata.tsv does not exist, exiting...

0.00	-TreeAnc: set-up

0.04	-###TEST.TreeAnc.prepare_tree: BEGIN
0.21	--ClockTree: Setting precision to level 2


MissingDataError: ERROR: ALMOST NO VALID DATE CONSTRAINTS